# Network analysis for coauthors - modularity

This looks at modularity of the main subgraph.

In [1]:
%load_ext autoreload
%autoreload 2

from src.util import load_coauthor_nx, ddir, fn_nodes, fn_statoids, fn_spp

import community
import networkx as nx
from operator import itemgetter
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import skbio.diversity.alpha as b
import math
import random

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from sklearn import preprocessing
import numpy as np

In [2]:
# Params
seed = 2021
random.seed(seed)

# Load graph
(G, nodes) = load_coauthor_nx() # abstracted into src

N nodes 389 ; N not nodes: 353
Proportion who did not coauthor 47.6 %

Name: 
Type: Graph
Number of nodes: 389
Number of edges: 508
Average degree:   2.6118


## Communities

In [3]:
# Read data
nodes_df = pd.read_csv(fn_nodes)
statoids = pd.read_csv(fn_statoids)
statoids = area_dict = dict(zip(statoids.DL, statoids.Country))

spp = pd.read_csv(fn_spp)[["idx", "full.name.of.describer"]]
spp = spp[spp.duplicated(subset="idx", keep=False)]             # keep only those with >1 authors

/home/ejysoh/miniconda3/envs/msc/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (22,30,60,61,65,86,89,106,117) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Add module information
communities = community.best_partition(G, random_state=seed)  # https://python-louvain.readthedocs.io/en/latest/api.html
nx.set_node_attributes(G, communities, 'modularity')

In [5]:
G.nodes["Michael Scott Engel"]

{'country_of_residence': 'US',
 'ns_spp_n': 224,
 'degree': 31,
 'betweenness': 0.15994704769615287,
 'eigenvector': 0.45957800061955284,
 'modularity': 8}

In [6]:
# Function to get highest eigenvector centrality from each module
# module_number refers to the original module number
def get_highest_eigen_in_module(module_number):

    module = [n for n in G.nodes() if G.nodes[n]['modularity'] == module_number]

    print("Number of nodes: ", len(set(module)), " in module ", module_number, "\n")

    # Then create a dictionary of the eigenvector centralities of those nodes
    module_eigenvector = {n:G.nodes[n]['eigenvector'] for n in module}

    # Then sort that dictionary and print the first 5 results
    module_sorted_by_eigenvector = sorted(module_eigenvector.items(), key=itemgetter(1), reverse=True)
    print("Modularity Sorted by Eigenvector Centrality:")
    for node in module_sorted_by_eigenvector[:10]:
        print(node[0], "| ", node[1], " | ", G.nodes[node[0]]['country_of_residence'])
        
    print("\n\n")

In [7]:
# Test the function on module 0
get_highest_eigen_in_module(0)

Number of nodes:  15  in module  0 

Modularity Sorted by Eigenvector Centrality:
Maximilian Schwarz |  0.08100179656093956  |  AU
Fritz Josef [Friedrich] Gusenleitner |  0.01636534386715893  |  AU
Karl Mazzucco |  0.012042369408000292  |  AU
Esther Ockermüller |  0.011238094664371935  |  AU
Jan Smit |  0.011238094664371935  |  NL
Klaus Standfuss |  0.009868848295510732  |  GA
Timofey Victorovich Levchenko |  0.009868848295510732  |  RS
Erwin Scheuchl |  0.006731571119070798  |  GM
Ardeshir Ariana |  0.0062603273188799125  |  JA
Gerald Hölzler |  0.00146737440266006  |  AU





In [8]:
# Create dictionary of module (key) and authors (value)

modularity = {}                            # Create a new, empty dictionary

for k, v in communities.items():           # Loop through the community dictionary
    if v not in modularity:
        modularity[v] = [k]                # Add a new key for a modularity class the code hasn't seen before
    else:
        modularity[v].append(k)            # Append a name to the list for a modularity class the code has already seen


# Create counter of countries (value) for each module (key)

countries = {}
counter = 0
for k, v in modularity.items():            # Loop through the new dictionary
    counter = counter + len(v)
    country_li = []
    for i in range(0, len(v)):
        country = nodes_df[nodes_df['full.name.of.describer'] == v[i]]['residence.country.describer'].values[0]
        country = str.split(country, "; ")
        country_li = country_li + [country[0]]
    countries[k] = Counter(country_li)

print("\n In total, ", counter, " authors/ nodes.")


 In total,  389  authors/ nodes.


In [9]:
# Count number of modules

print(len(modularity), " modules")    # number of modules

57  modules


In [10]:
# Get a dataframe of module (index) and subgraph (0 column)

subgraphs = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]

def get_subgraph(node_name):
    subgraph_idx = -1
    
    for i in range(0, len(subgraphs)):
        if (node_name in list(subgraphs[i])):
            subgraph_idx = i
    
    return subgraph_idx
          
community_subgraph = pd.DataFrame.from_dict({key:get_subgraph(value[0]) for (key,value) in modularity.items()}, orient='index')

In [11]:
# Get number of species for each module as a dataframe

N_species = {}

for i, sg in enumerate(modularity.items()):
    spp_li = []
    
    for auth in sg[1]:
        spp_li = spp_li + list(spp[spp['full.name.of.describer']==auth]['idx'].values)
        
    N_species[i] = len(set(spp_li))
    
N_species = pd.DataFrame.from_dict(N_species, orient="index")
N_species.columns = ["N_species"]

In [12]:
# Convert dictionary countries to pandas.DataFrame countries_df
countries_df = pd.DataFrame.from_dict(countries, orient='index')
cols = list(countries_df.columns.values) 

# Shift column [unknown] to end
cols.pop(cols.index('[unknown]'))
cols_rearranged = cols + ['[unknown]']
countries_df = countries_df[cols_rearranged]

# Sort by index
countries_df = countries_df.sort_index()

In [13]:
# Get number of authors and countries as a dataframe for each module

countries_summary1 = countries_df.sum(axis=1)   # count number of authors in each module
countries_summary2 = countries_df.count(axis=1) # count number of countries in each module

# Combine all dataframes (N authors, N countries, Subgraph, N species)

countries_summary = pd.concat([countries_summary1, countries_summary2], axis=1) 
countries_summary = countries_summary.merge(community_subgraph, "outer", left_index=True, right_index=True)
countries_summary = countries_summary.merge(N_species, "outer", left_index=True, right_index=True)
countries_summary.columns = ['N_authors', 'N_countries', 'Subgraph_id', 'N_species']
countries_summary.index.names = ['Modules']

# Create a new column called "idx" (which will be the way the dataframe is sorted later for labels)

# It is sorted by Subgraph_id and N_authors
countries_summary = countries_summary.sort_values(['Subgraph_id', 'N_authors'], ascending=[True,  False])
countries_summary['idx'] = range(0, len(countries_summary))
# Create label
countries['lab'] = ' [' + countries_summary['N_authors'].astype(int).astype(str) + ',' +\
    countries_summary['N_countries'].astype(int).astype(str) + "," +\
    countries_summary['N_species'].astype(int).astype(str) + '] ' +\
    "id" + countries_summary.idx.astype(str).str.pad(2, "left", "0") + " / " +\
    "S" + countries_summary['Subgraph_id'].astype(str).str.pad(2, "left", "0")

# Sort by index and save idx
countries_summary = countries_summary.sort_index()
idx = countries_summary.idx.values
countries_summary = countries_summary.sort_values('idx')

In [14]:
# Summary stats for ALL subgraphs
countries_summary.agg({
    'N_authors': ['median', 'min', 'max'],
    'N_countries': ['median', 'min', 'max']
})

,N_authors,N_countries
median,2.0,1.0
min,2.0,1.0
max,49.0,12.0


In [15]:
# Summary stats for subgraph 1
countries_summary.groupby('Subgraph_id')\
                 .agg({'N_authors': ['median', 'min', 'max'],
                       'N_countries': ['median', 'min', 'max', 'count']})\
                 .iloc[0] # for subgraph 1

N_authors    median    16.0
             min        5.0
             max       49.0
N_countries  median     5.0
             min        1.0
             max       12.0
             count     13.0
Name: 0, dtype: float64

All of the subgraphs only have one cluster, but the largest one has 12 clusters.

In [16]:
# Get proportions
countries_prop = countries_df.apply(lambda r: round(r/r.sum()*100, 1), axis=1)
countries_prop.index = idx
countries_prop = countries_prop.sort_index()
countries_prop.index = countries['lab']

In [17]:
# Get full country name from abbreviated form
def parse_countries(country):
    if country == "[unknown]":
        return country
    else:
        return statoids[country]

countries_prop.columns = [parse_countries(x) for x in countries_prop.columns]
# countries_prop.columns = countries_prop.columns + " [" + countries_prop.count().astype(str) + "]"

In [18]:
countries_prop.columns

Index(['Algeria', 'Austria', 'Gambia', 'Germany', 'Israel', 'Japan',
       'Netherlands', 'Russian Federation', 'Turkey', 'Argentina', 'Chile',
       'Spain', 'United States of America', 'Australia', 'Belgium', 'Canada',
       'Cameroon', 'Ethiopia', 'France', 'New Zealand', 'South Africa',
       'Thailand', 'China', 'Indonesia', 'Norway', 'Sweden', 'Taiwan',
       'Brazil', 'Colombia', 'Denmark', 'Czech Republic', 'Mexico', 'Peru',
       'Saudi Arabia', 'United Kingdom', 'Costa Rica', 'India', 'Pakistan',
       'Panama', 'Italy', 'Switzerland', 'Paraguay', 'Cuba', 'Kenya',
       'Portugal', 'Ukraine', 'Iran', 'Korea, South', 'Venezuela',
       '[unknown]'],
      dtype='object')

In [ ]:
# Plot graph
countries_prop_t = countries_prop.transpose(copy=True)
countries_prop_t[countries_prop_t.columns] = countries_prop_t[countries_prop_t.columns].replace({0: float('nan')})
plt.figure(figsize=(15, 13))
ax = sns.heatmap(countries_prop_t, cmap=plt.get_cmap("Spectral_r"), linecolor="k", linewidths=0.1)
ax.set(xlabel='\nProportion of country from each module (%)', ylabel='Countries')

In [ ]:
countries_prop.iloc[3][countries_prop.iloc[3].notnull()]

In [ ]:
# Calculate diversity indices

indices_simpsons = []
indices_shannon = []

for i, idx in enumerate(countries_prop.index):
    country_counts = [x for x in countries_prop.iloc[i].values if not math.isnan(x)]
    
    # indices = countries_prop.iloc[i][countries_prop.iloc[i].notna()].values
    indices_simpsons =  indices_simpsons + [b.simpson(country_counts)]
    indices_shannon =  indices_shannon + [b.shannon(country_counts)]

countries_summary['simpson'] = indices_simpsons
countries_summary['shannon'] = indices_shannon

In [ ]:
countries_summary_original = countries_summary.copy(deep=True)
countries_summary_original

In [ ]:
# countries_summary = countries_summary_original.copy(deep=True)

In [ ]:
# Scale values by min max for PCA

cols = ['N_authors', 'N_countries', 'N_species', 'simpson']
x = countries_summary[cols].values #returns a numpy array
scaler = preprocessing.StandardScaler()
countries_summary_scaled = pd.DataFrame(scaler.fit_transform(x))

In [ ]:
# Perform PCA

pca = PCA(n_components=2, svd_solver='full')
pca.fit(countries_summary_scaled)
pca.values = pca.transform(countries_summary_scaled)
pca.values = pd.DataFrame(pca.values, columns=['PC1', 'PC2'])

var = pca.explained_variance_ratio_
countries_summary = countries_summary.reset_index().merge(pca.values, left_index=True, right_index=True)

In [ ]:
# Perform DBSCAN
cluster = KMeans(n_clusters=5, random_state=22).fit(countries_summary[['PC1', 'PC2']].values)
countries_summary['KMeans group'] = cluster.labels_ + 1
countries_summary.loc[countries_summary['KMeans group'] == 0, 'KMeans group'] = "Ungrouped"

In [ ]:
countries_summary

In [ ]:
# Get the PCA components (loadings)
PCs = pca.components_

# Use quiver to generate the basic plot
fig = plt.figure(figsize=(8, 8))
plt.quiver(np.zeros(PCs.shape[1]), np.zeros(PCs.shape[1]),
           PCs[0,:]*4, PCs[1,:]*3, 
           angles='xy', scale_units='xy', scale=1, 
           width = 0.001, headwidth = 20, headlength = 20)

# Add labels based on feature names (here just numbers)
feature_names = ['N authors', 'N countries', 'N species', 'Simpsons']
for i,j,z in zip(PCs[1,:]*2, PCs[0,:]*3, feature_names):
    plt.text(j, i, z, ha='center', va='center')

# # Add unit circle
# circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
# plt.gca().add_artist(circle)

# Ensure correct aspect ratio and axis limits
# plt.axis('equal')
plt.xlim([countries_summary['PC1'].min()-1, countries_summary['PC1'].max()+1])
plt.ylim([countries_summary['PC2'].min()-1, countries_summary['PC2'].max()+1])

# Plot value
n_colours = len(countries_summary['KMeans group'].unique())
sns.scatterplot(countries_summary['PC1'], countries_summary['PC2'],  
                hue=countries_summary['KMeans group'], palette=sns.color_palette("hls", n_colours), legend="brief")

# Label axes
xlab = "PC 1 [" + str(round(var[0]*100, 1)) + "%]"
plt.xlabel(xlab)
ylab = "PC 2 [" + str(round(var[1]*100, 1)) + "%]"
plt.ylabel(ylab)

plt.show()

In [ ]:
countries_summary.agg({'N_authors': ['count', 'median', 'min', 'max'],
                       'N_countries': ['median', 'min', 'max'],
                       'N_species': ['median', 'min', 'max'],
                       'simpson': ['median', 'min', 'max']})

In [ ]:
# Summary stats for subgraph 1
countries_summary.groupby('KMeans group')\
                 .agg({'N_authors': ['count', 'median', 'min', 'max'],
                       'N_countries': ['median', 'min', 'max'],
                       'N_species': ['median', 'min', 'max'],
                       'simpson': ['median', 'min', 'max']})

# group 1: high authors, low countries, high species, low simpsons = not international, contribute to many species
# group 2: mid authors, mid countries, mid species, high simpsons = international, mid-sized
# group 3: low authors, low countries, low species, mid simpsons = international, small-sized
# group 4: high authors, high countries, high species, high simpsons = international, big-sized
# group 5: low authors, low countries, low species, low simpsons = not international, contribute to few species

In [ ]:
countries_summary.iloc[3]

In [ ]:
subgraphs_subset = countries_summary[(countries_summary.N_countries>=3) & (countries_summary.Subgraph_id==0)]

spp_li = []
counter = 0
for i, sg in enumerate(subgraphs_subset.index.values):
    for auth in modularity[sg]:
        spp_li = spp_li + list(spp[spp['full.name.of.describer']==auth]['idx'].values)
    counter += 1

spp_li = len(set(spp_li))
n_auth = subgraphs_subset['N_authors'].sum()
from_subgraphs = subgraphs_subset['Subgraph_id'].unique()

print(
    "Authors with >=3 countries accounted for", spp_li, "species for ", counter, " clusters/modules and with", 
    n_auth, "authors from subgraphs", from_subgraphs, "( n=", len(from_subgraphs), ")."
)

In [ ]:
# Clusters with only 2 countries
print(len(countries_summary[countries_summary.N_countries==2]))
# Clusters with only 1 country
print(len(countries_summary[countries_summary.N_countries==1]))
# Clusters with only 1 country and 2 authors
print(len(countries_summary[(countries_summary.N_countries==1) & (countries_summary.N_authors <=2)]))

In [ ]:
print(countries_summary[countries_summary['KMeans group']==1], "\n")

# Brazil group and US/New Zealand group ("old")
print(modularity[9], "\n")
print(modularity[10])

In [ ]:
countries_df_m2 = countries_df.loc[countries_df.count(axis=1) > 2] # countries with <=2
print(countries_df_m2.count(axis=0).sort_values(ascending=False)[0:10])

countries_df_l2 = countries_df.loc[countries_df.count(axis=1) < 2] # countries with <=2
print(countries_df_l2.count(axis=0).sort_values(ascending=False)[0:10])

In [ ]:
countries_summary.sort_values('idx')

In [ ]:
plt.figure(figsize=(8, 8))
ax = sns.regplot(countries_summary['N_countries'], countries_summary['N_authors'])
ax.set(xlabel='\n Number of countries', 
       ylabel='Number of authors')

In [ ]:
[get_highest_eigen_in_module(x) for x in countries_summary[countries_summary.N_authors>=10].index.values]